In [ ]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
from torch.utils.data import DataLoader, Dataset
import torch
from data_import import *

In [2]:
oregon_data_dict = oregon_import()

file import: 100%|██████████| 3/3 [00:00<00:00,  4.21it/s]


In [ ]:
wa_df = oregon_data_dict['train_timeseries'].iloc[4:,:].reset_index(inplace=False, drop=True)
wa_df = wa_df[wa_df['fips']==41067]

In [ ]:
X_1 = wa_df.iloc[:7, (wa_df.columns != 'score') & (wa_df.columns != 'date')]
y_1 = wa_df.iloc[6, wa_df.columns == 'score']
date_1 = wa_df.iloc[:7, wa_df.columns == 'date']

In [ ]:
X_2 = wa_df.iloc[7:14, (wa_df.columns != 'score') & (wa_df.columns != 'date')]
y_2 = wa_df.iloc[13, wa_df.columns == 'score']
date_2 = wa_df.iloc[7:14, wa_df.columns == 'date']

In [9]:
class DroughtDataset(Dataset):
    """
    Readying Drought dataset for model.
    """
    def __init__(self, df):
        self.X = torch.tensor(df.iloc[:, (df.columns != 'score') & (df.columns != 'date')].values)
        self.y = torch.tensor(df.iloc[:, df.columns == 'score'].dropna().values)
        self.date = np.array(df.iloc[:, df.columns == 'date'].values)

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        output_X = self.X[7*index:7+7*index]
        output_y = self.y[6 + 7*index]
        output_date = self.date[7*index:7+7*index]

        return output_X, output_y, output_date

In [ ]:
wa_dataset = DroughtDataset(wa_df)

In [13]:
import torch.nn as nn
torch.manual_seed(1)

In [ ]:
class LSTM_Model(nn.Module)
    def __init__(self, *args, **kwargs):
        super(LSTM_Model, self).__init__()